In [1]:
import os
os.environ['STPSF_PATH'] = '/blue/adamginsburg/t.yoo/from_red/stpsf-data'

print("Starting crowdsource_catalogs_long", flush=True)
print(f"os.environ.get: {os.environ.get('STPSF_PATH')}", flush=True)
print(f"os.getenv: {os.getenv('STPSF_PATH')}", flush=True)

# Import stpsf first, then immediately override its config
import stpsf as webbpsf
# Force the path to be set directly in the config
webbpsf.conf.STPSF_PATH = '/blue/adamginsburg/t.yoo/from_red/stpsf-data'

# Continue with other imports
from webbpsf.utils import to_griddedpsfmodel
from jwst.datamodels import dqflags

# Continue with other imports
from webbpsf.utils import to_griddedpsfmodel
import glob

import time
import numpy
import crowdsource
import regions
import numpy as np
from functools import cache
from astropy.convolution import convolve, Gaussian2DKernel, interpolate_replace_nans, convolve_fft
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.visualization import simple_norm
from astropy.modeling.fitting import LevMarLSQFitter
from astropy import wcs
from astropy import table
from astropy import stats
from astropy import units as u
from astropy.nddata import NDData
from astropy.io import fits
from scipy import ndimage
import requests
import requests.exceptions
import urllib3
import urllib3.exceptions
from photutils.detection import DAOStarFinder, IRAFStarFinder
from photutils.psf import IntegratedGaussianPRF, extract_stars, EPSFStars, EPSFModel
try:
    # version >=1.7.0, doesn't work: the PSF is broken (https://github.com/astropy/photutils/issues/1580?)
    from photutils.psf import PSFPhotometry, IterativePSFPhotometry, SourceGrouper
except:
    # version 1.6.0, which works
    from photutils.psf import BasicPSFPhotometry as PSFPhotometry, IterativelySubtractedPSFPhotometry as IterativePSFPhotometry, DAOGroup as SourceGrouper
try:
    from photutils.background import MMMBackground, MADStdBackgroundRMS, MedianBackground, Background2D, LocalBackground
except:
    from photutils.background import MMMBackground, MADStdBackgroundRMS, MedianBackground, Background2D
    from photutils.background import MMMBackground as LocalBackground

from photutils.psf import EPSFBuilder
from photutils.psf import extract_stars

import warnings
from astropy.utils.exceptions import AstropyWarning, AstropyDeprecationWarning
warnings.simplefilter('ignore', category=AstropyWarning)
warnings.simplefilter('ignore', category=AstropyDeprecationWarning)

from crowdsource import crowdsource_base
from crowdsource.crowdsource_base import fit_im, psfmod

from astroquery.svo_fps import SvoFps
from astropy.table import Table, vstack

import pylab as pl
pl.rcParams['figure.facecolor'] = 'w'
pl.rcParams['image.origin'] = 'lower'

import os
print("Importing webbpsf", flush=True)
print(f"STPSF_PATH before stpsf import: {os.environ.get('STPSF_PATH')}", flush=True)

#import stpsf as webbpsf
#print(f"Webbpsf version: {webbpsf.__version__}")
#from webbpsf.utils import to_griddedpsfmodel
import datetime
print("Done with imports", flush=True)
import sys

def print(*args, **kwargs):
    now = datetime.datetime.now().isoformat()
    from builtins import print as printfunc
    return printfunc(f"{now}:", *args, **kwargs)


class WrappedPSFModel(crowdsource.psf.SimplePSF):
    """
    wrapper for photutils GriddedPSFModel
    """
    def __init__(self, psfgridmodel, stampsz=19):
        self.psfgridmodel = psfgridmodel
        self.default_stampsz = stampsz

    def __call__(self, col, row, stampsz=None, deriv=False):

        if stampsz is None:
            stampsz = self.default_stampsz

        parshape = numpy.broadcast(col, row).shape
        tparshape = parshape if len(parshape) > 0 else (1,)

        # numpy uses row, column notation
        rows, cols = np.indices((stampsz, stampsz)) - (np.array([stampsz, stampsz])-1)[:, None, None] / 2.

        # explicitly broadcast
        col = np.atleast_1d(col)
        row = np.atleast_1d(row)
        #rows = rows[:, :, None] + row[None, None, :]
        #cols = cols[:, :, None] + col[None, None, :]

        # photutils seems to use column, row notation
        # only works with photutils <= 1.6.0 - but is wrong there
        #stamps = self.psfgridmodel.evaluate(cols, rows, 1, col, row)
        # it returns something in (nstamps, row, col) shape
        # pretty sure that ought to be (col, row, nstamps) for crowdsource

        # andrew saydjari's version here:
        # it returns something in (nstamps, row, col) shape
        stamps = []
        for i in range(len(col)):
            # the +0.5 is required to actually center the PSF (empirically)
            #stamps.append(self.psfgridmodel.evaluate(cols+col[i]+0.5, rows+row[i]+0.5, 1, col[i], row[i]))
            # the above may have been true when we were using (incorrectly) offset PSFs
            stamps.append(self.psfgridmodel.evaluate(cols+col[i], rows+row[i], 1, col[i], row[i]))

        stamps = np.array(stamps)

        # for oversampled stamps, they may not be normalized
        stamps /= stamps.sum(axis=(1,2))[:,None,None]
        # this is evidently an incorrect transpose
        #stamps = np.transpose(stamps, axes=(0,2,1))

        if deriv:
            dpsfdrow, dpsfdcol = np.gradient(stamps, axis=(1, 2))

        ret = stamps
        if parshape != tparshape:
            ret = ret.reshape(stampsz, stampsz)
            if deriv:
                dpsfdrow = dpsfdrow.reshape(stampsz, stampsz)
                dpsfdcol = dpsfdcol.reshape(stampsz, stampsz)
        if deriv:
            ret = (ret, dpsfdcol, dpsfdrow)

        return ret

    def render_model(self, col, row, stampsz=None):
        """
        this function likely does nothing?
        """
        if stampsz is not None:
            self.stampsz = stampsz

        rows, cols = np.indices(self.stampsz, dtype=float) - (np.array(self.stampsz)-1)[:, None, None] / 2.

        return self.psfgridmodel.evaluate(cols, rows, 1, col, row).T.squeeze()

def load_data(filename):
    fh = fits.open(filename)
    im1 = fh
    data = im1['SCI'].data
    try:
        wht = im1['WHT'].data
    except KeyError:
        wht = None
    err = im1['ERR'].data
    instrument = im1[0].header['INSTRUME']
    telescope = im1[0].header['TELESCOP']
    obsdate = im1[0].header['DATE-OBS']
    return fh, im1, data, wht, err, instrument, telescope, obsdate
def get_psf_model(filtername, proposal_id, field,
                  module,
                  use_webbpsf=False,
                  obsdate=None,
                  use_grid=False,
                  blur=False,
                  target='brick',
                  stampsz=19,
                  oversample=1,
                  basepath='/orange/adamginsburg/jwst/'):
    """
    Return two types of PSF model, the first for DAOPhot and the second for Crowdsource
    """

    basepath = f'{basepath}/{target}'

    blur_ = "_blur" if blur else ""

    # psf_fn = f'{basepath}/{instrument.lower()}_{filtername}_samp{oversample}_nspsf{npsf}_npix{fov_pixels}.fits'
    # if os.path.exists(str(psf_fn)):
    #     # As a file
    #     print(f"Loading grid from psf_fn={psf_fn}", flush=True)
    #     grid = to_griddedpsfmodel(psf_fn)  # file created 2 cells above
    #     if isinstance(big_grid, list):
    #         print(f"PSF IS A LIST OF GRIDS!!! this is incompatible with the return from nrc.psf_grid")
    #         grid = grid[0]

    if use_webbpsf:
        #with open(os.path.expanduser('~/.mast_api_token'), 'r') as fh:
        #    api_token = fh.read().strip()
        #from astroquery.mast import Mast

        #for ii in range(10):
        #    try:
        #        Mast.login(api_token.strip())
        #        break
        #    except (requests.exceptions.ReadTimeout, urllib3.exceptions.ReadTimeoutError, TimeoutError) as ex:
        #        print(f"Attempt {ii} to log in to MAST: {ex}")
        #        time.sleep(5)
        #os.environ['MAST_API_TOKEN'] = api_token.strip()

        has_downloaded = False
        ntries = 0
        while not has_downloaded:
            ntries += 1
            try:
                print("Attempting to download WebbPSF data", flush=True)
                if filtername.upper() in ['F140M', 'F150W', 'F162M', 'F164N', 'F182M', 'F187N',
                                  'F200W', 'F210M', 'F212N', 'F250M', 'F300M', 'F322W2',
                                  'F335M', 'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 'F444W',
                                  'F460M', 'F466N', 'F480M']:
                    nrc = webbpsf.NIRCam()
                else:
                    nrc = webbpsf.MIRI()
                nrc.load_wss_opd_by_date(f'{obsdate}T00:00:00')
                nrc.filter = filtername
                if module in ('nrca', 'nrcb'):
                    if 'F4' in filtername.upper() or 'F3' in filtername.upper():
                        nrc.detector = f'{module.upper()}5' # I think NRCA5 must be the "long" detector?
                    else:
                        nrc.detector = f'{module.upper()}1' #TODO: figure out a way to use all 4?
                    # default oversampling is 4
                    grid = nrc.psf_grid(num_psfs=16, all_detectors=False, verbose=True, save=True)
                elif 'mirimage' in module:
                    print('module', module, flush=True)
                    print(nrc.detector)
                    nrc.detector = 'MIRIM'
                    grid = nrc.psf_grid(num_psfs=16, all_detectors=False, verbose=True, save=True)
                else:
                    grid = nrc.psf_grid(num_psfs=16, all_detectors=True, verbose=True, save=True)
                has_downloaded = True
            except (urllib3.exceptions.ReadTimeoutError, requests.exceptions.ReadTimeout, requests.HTTPError) as ex:
                print(f"Failed to build PSF: {ex}", flush=True)
            except Exception as ex:
                print(ex, flush=True)
                if ntries > 10:
                    # avoid infinite loops
                    raise ValueError("Failed to download PSF, probably because of an error listed above")
                else:
                    continue

        if use_grid:
            if isinstance(grid, list):
                grid = grid[0]
            return grid, WrappedPSFModel(grid, stampsz=stampsz)
        else:
            # there's no way to use a grid across all detectors.
            # the right way would be to use this as a grid of grids, but that apparently isn't supported.
            if isinstance(grid, list):
                grid = grid[0]

            #yy, xx = np.indices([31,31], dtype=float)
            #grid.x_0 = grid.y_0 = 15.5
            #psf_model = crowdsource.psf.SimplePSF(stamp=grid(xx,yy))

            # bigger PSF probably needed
            yy, xx = np.indices([61, 61], dtype=float)
            grid.x_0 = grid.y_0 = 30
            psf_model = crowdsource.psf.SimplePSF(stamp=grid(xx, yy))

            return grid, psf_model
    else:

        grid = psfgrid = to_griddedpsfmodel(f'{basepath}/psfs/{filtername.upper()}_{proposal_id}_{field}_merged_PSFgrid_oversample{oversample}{blur_}.fits')

        # if isinstance(grid, list):
        #     print(f"Grid is a list: {grid}")
        #     psf_model = WrappedPSFModel(grid[0])
        #     dao_psf_model = grid[0]
        # else:

        psf_model = WrappedPSFModel(grid, stampsz=stampsz)
        dao_psf_model = grid

        return grid, psf_model
def get_uncertainty(err, data, dq=None, wht=None):

    if dq is None:
        dq = np.zeros(data.shape, dtype='int')

    # crowdsource uses inverse-sigma, not inverse-variance
    weight = err**-1
    #maxweight = np.percentile(weight[np.isfinite(weight)], 95)
    #minweight = np.percentile(weight[np.isfinite(weight)], 5)
    #badweight =  np.percentile(weight[np.isfinite(weight)], 1)
    #weight[err < 1e-5] = 0
    #weight[(err == 0) | (wht == 0)] = np.nanmedian(weight)
    #weight[np.isnan(weight)] = 0
    bad = np.isnan(weight) | (data == 0) | np.isnan(data) | (weight == 0) | (err == 0)
    #if dq is not None:
    #    # only 0 is OK
    #    bad |= (dq != 0)
    if wht is not None:
        bad |= (wht == 0)

    #weight[weight > maxweight] = maxweight
    #weight[weight < minweight] = minweight
    # it seems that crowdsource doesn't like zero weights
    # may have caused broked f466n? weight[bad] = badweight
    #weight[bad] = minweight
    # crowdsource explicitly handles weight=0, so this _should_ work.
    weight[bad] = 0

    # Expand bad pixel zones for dq
    #bad_for_dq = ndimage.binary_dilation(bad, iterations=2)
    #dq[bad_for_dq] = 2 | 2**30 | 2**31
    #print(f"Total bad pixels = {bad.sum()}, total bad for dq={bad_for_dq.sum()}")

    return dq, weight, bad

def save_photutils_results(result, ww, filename,
                           im1, detector,
                           basepath, filtername, module, desat, bgsub, exposure_, visitid_, vgroupid_,
                           psf=None,
                           blur=False,
                           basic_or_iterative='basic',
                           each_exposure=True,
                           proposal_id='6151',
                           epsf_="",
                           group="",
                           fpsf=""):
    print("Saving photutils results.")
    blur_ = "_blur" if blur else ""

    pixscale = (ww.proj_plane_pixel_area()**0.5).to(u.arcsec)
    if 'x_fit' in result.colnames:
        coords = ww.pixel_to_world(result['x_fit'], result['y_fit'])
        result['skycoord_centroid'] = coords
    elif 'xcentroid' in result.colnames:
        coords = ww.pixel_to_world(result['xcentroid'], result['ycentroid'])
        result['skycoord_centroid'] = coords
    elif 'x_init' in result.colnames:
        coords = ww.pixel_to_world(result['x_init'], result['y_init'])
        result['skycoord_init'] = coords
    else:
        raise KeyError(f"No x value found in {result.colnames}")
    print(f'len(result) = {len(result)}, len(coords) = {len(coords)}, type(result)={type(result)}', flush=True)
    detector = "" # no detector #'s for long
    if each_exposure:
        result.meta['exposure'] = exposure_
    if visitid_ is not None:
        #result.meta['visit'] = int(visitid_[-3:]) if visitid_ is not '' else None
        result.meta['visit'] = visitid_[-3:] if visitid_ is not '' else None
    if vgroupid_ is not None:
        result.meta['vgroup'] = vgroupid_[-4:] if vgroupid_ is not '' else None
        
    result.meta['filename'] = filename
    result.meta['filter'] = filtername
    result.meta['module'] = module
    result.meta['detector'] = detector
    result.meta['pixscale'] = pixscale.to(u.deg).value
    result.meta['pixscale_as'] = pixscale.to(u.arcsec).value
    result.meta['proposal_id'] = proposal_id

    if 'RAOFFSET' in im1[0].header:
        result.meta['RAOFFSET'] = im1[0].header['RAOFFSET']
        result.meta['DEOFFSET'] = im1[0].header['DEOFFSET']
    elif 'RAOFFSET' in im1[1].header:
        result.meta['RAOFFSET'] = im1[1].header['RAOFFSET']
        result.meta['DEOFFSET'] = im1[1].header['DEOFFSET']

    if 'x_err' in result.colnames:
        result['dra'] = result['x_err'] * pixscale
        result['ddec'] = result['y_err'] * pixscale
    #jw06151002001_02101_00001_mirimage_i2d.fits
    tblfilename = f"{basepath}/{filtername}/{filtername.lower()}_{module}{detector}{visitid_}{vgroupid_}{exposure_}{desat}{bgsub}{epsf_}{blur_}{group}_daophot_{basic_or_iterative}.fits"

    result.write(tblfilename, overwrite=True)

    print("tblfilename={tblfilename}, filename={filename}, suffix={suffix}, filtername={filtername}, module={module}, desat={desat}, bgsub={bgsub}, fpsf={fpsf} blur={blur}")

    result.write(tblfilename, overwrite=True)
    print(f"Completed {basic_or_iterative} photometry, and wrote out file {tblfilename}")

    return result
proposal_id = '6151'
field='001'
module='nrcalong'
use_webbpsf = True

basepath = f'/orange/adamginsburg/jwst/w51/'
filename = basepath + 'F405N/pipeline/jw06151001001_03101_00001_nrcalong_cal.fits'
filtername='F405N'
hdu = fits.open(filename)
fwhm_tbl = Table.read(f'{basepath}/reduction/fwhm_table.ecsv')
row = fwhm_tbl[fwhm_tbl['Filter'] == filtername]
fwhm = fwhm_arcsec = float(row['PSF FWHM (arcsec)'][0])
fwhm_pix = float(row['PSF FWHM (pixel)'][0])

# redundant, saves me renaming variables....
filt = 'F405N'

# file naming suffixes
desat = ''
bgsub = ''
epsf_ = ""
#exposure_ = f'_exp{exposurenumber:05d}' if exposurenumber is not None else ''
exposure_ = '_exp00001'
#visitid_ = f'_visit{int(visit_id):03d}' if visit_id is not None else ''
#visitid_ = f'_visit{visit_id}' if visit_id is not None else ''
visitid_ = '_visit001'
#vgroupid_ = f'_vgroup{vgroup_id}' if vgroup_id is not None else ''
vgroupid_='_vgroup03101'
blur_ = ""
group = ""

print(f"Starting cataloging on {filename}", flush=True)
fh, im1, data, wht, err, instrument, telescope, obsdate = load_data(filename)

# set up coordinate system
ww = wcs.WCS(im1[1].header)
pixscale = ww.proj_plane_pixel_area()**0.5
cen = ww.pixel_to_world(im1[1].shape[1]/2, im1[1].shape[0]/2)
data = data.astype('float32')

# Load PSF model
grid, psf_model = get_psf_model(filtername, proposal_id, field,
                                module=module,
                                use_webbpsf=use_webbpsf,
                                # if we're doing each exposure, we want the full grid
                                use_grid=True,
                                blur=False,
                                target='w51',
                                obsdate=obsdate,
                                basepath='/orange/adamginsburg/jwst/')
dao_psf_model = grid

# bound the flux to be >= 0 (no negative peak fitting)
#dao_psf_model.flux.min = 0

dq, weight, bad = get_uncertainty(err, data, wht=wht, dq=im1['DQ'].data if 'DQ' in im1 else None)

filter_table = SvoFps.get_filter_list(facility=telescope, instrument=instrument)
filter_table.add_index('filterID')
if filtername.upper() in ['F140M', 'F150W', 'F162M', 'F164N', 'F182M', 'F187N',
                        'F200W', 'F210M', 'F212N', 'F250M', 'F300M', 'F322W2',
                        'F335M', 'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 'F444W',
                        'F460M', 'F466N', 'F480M']:
    instrument = 'NIRCam'
else:
    instrument = 'MIRI'
eff_wavelength = filter_table.loc[f'{telescope}/{instrument}.{filt}']['WavelengthEff'] * u.AA

# DAO Photometry setup
grouper = SourceGrouper(2 * fwhm_pix)
mmm_bkg = MMMBackground()

#filtered_errest = stats.sigma_clipped_stats(data, stdfunc='mad_std')
filtered_errest = np.nanmedian(err)
print(f'Error estimate for DAO from median(err): {filtered_errest}', flush=True)

nsigma=5.0

daofind_tuned = DAOStarFinder(threshold=nsigma * filtered_errest,
                                fwhm=fwhm_pix, roundhi=1.0, roundlo=-1.0,
                                sharplo=0.30, sharphi=1.40)
#daofind_tuned = DAOStarFinder(threshold=4 * filtered_errest,
#                              fwhm=fwhm_pix, roundhi=0.8, roundlo=-0.9,
#                              sharplo=0.25, sharphi=1.20)
print("Finding stars with daofind_tuned", flush=True)

# empirically determined in debugging session with Taehwa on 2025-12-09:
# with just nan_to_num, setting pixels to zero, some stars got "erased"
kernel = Gaussian2DKernel(x_stddev=fwhm_pix/2.355)
mask = np.isnan(data)
if 'DQ' in im1:
    dqarr = im1['DQ'].data
    is_saturated = (dqarr & dqflags.pixel['SATURATED']) != 0
    # we want original data_ to be untouched for imshowing diagnostics etc.
    data_ = data.copy()
    data_[is_saturated] = np.nan
    mask |= is_saturated

    # also mask the edge of the image
    edge_width = 10
    mask[:edge_width, :] = True
    mask[-edge_width:, :] = True
    mask[:, :edge_width] = True
    mask[:, -edge_width:] = True
else:
    data_ = data

nan_replaced_data = interpolate_replace_nans(data_, kernel, convolve=convolve_fft)

# get the number of nan pixels for nan_replcaed_data
print(f"Number of NaN pixels in data: {np.isnan(data).sum()}", flush=True)
print(f"Number of NaN pixels in nan_replaced_data: {np.isnan(nan_replaced_data).sum()}", flush=True)

finstars = daofind_tuned(nan_replaced_data,
                            mask=mask)

phot_basic = PSFPhotometry(finder=daofind_tuned,#finder_maker(),
                             #grouper=grouper, # the grouper is needed to jointly fit stars and avoid faint stars jumping onto bright ones
                             grouper=None,
                             # localbkg_estimator=None, # must be none or it un-saturates pixels
                             localbkg_estimator=LocalBackground(5, 15),
                             psf_model=dao_psf_model,
                             fitter=LevMarLSQFitter(),
                             fit_shape=(5, 5),
                             aperture_radius=2*fwhm_pix,
                             progress_bar=True,
                            )

print("About to do BASIC photometry....")
result = phot_basic(nan_replaced_data, mask=mask)
# I want to use daofind params in the future
result['roundness1'] = finstars['roundness1']
result['roundness2'] = finstars['roundness2']
result['sharpness'] = finstars['sharpness']
print(f"Done with BASIC photometry.  len(result)={len(result)} dt={time.time() - t0}")

# remove negative-peak and zero-peak sources (they affect the residuals badly)
# we don't want to remove them now; we need to flag out objects that are too close to negative stars
#bad = result['flux_fit'] <= 0
#result = result[~bad]

result = save_photutils_results(result, ww, filename,
                                im1=im1, detector=detector,
                                basepath=basepath,
                                filtername=filtername, module=module,
                                desat=desat, bgsub=bgsub,
                                blur=False,
                                exposure_=exposure_,
                                visitid_=visitid_,
                                vgroupid_=vgroupid_,
                                basic_or_iterative='basic',
                                
                                epsf_=epsf_,
                                group=group,
                                psf=None)


Starting crowdsource_catalogs_long
os.environ.get: /blue/adamginsburg/t.yoo/from_red/stpsf-data
os.getenv: /blue/adamginsburg/t.yoo/from_red/stpsf-data


<>:359: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:361: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:359: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:361: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/scratch/local/23155831/ipykernel_819749/951555797.py:14: DeprecationWarning: "
        This message is for information only and WebbPSF will continue to function as normal.
        The WebbPSF library has been moved/renamed to STPSF.
        Please see https://stpsf.readthedocs.io/en/stable/ for more information.
        WebbPSF is now an alias of STPSF and is running code from the STPSF library.
        
  from webbpsf.utils import to_griddedpsfmodel


Importing webbpsf
STPSF_PATH before stpsf import: /blue/adamginsburg/t.yoo/from_red/stpsf-data
Done with imports
2026-01-23T16:27:22.720003: Starting cataloging on /orange/adamginsburg/jwst/w51/F405N/pipeline/jw06151001001_03101_00001_nrcalong_cal.fits
2026-01-23T16:27:22.760015: Attempting to download WebbPSF data
/blue/adamginsburg/t.yoo/from_red/stpsf-data
path /blue/adamginsburg/t.yoo/from_red/stpsf-data


/scratch/local/23155831/ipykernel_819749/951555797.py:359: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  result.meta['visit'] = visitid_[-3:] if visitid_ is not '' else None
/scratch/local/23155831/ipykernel_819749/951555797.py:361: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  result.meta['vgroup'] = vgroupid_[-4:] if vgroupid_ is not '' else None


iterating query, tdelta=3.0
iterating query, tdelta=6.0

MAST OPD query around UTC: 2025-05-06T00:00:00.000
                        MJD: 60801.0

OPD immediately preceding the given datetime:
	URI:	 mast:JWST/product/R2025050102-NRCA1_FP6-1.fits
	Date (MJD):	 60796.3752
	Delta time:	 -4.6248 days

OPD immediately following the given datetime:
	URI:	 mast:JWST/product/O2025050601-NRCA1_FP6-1.fits
	Date (MJD):	 60801.1320
	Delta time:	 0.1320 days
User requested choosing OPD time closest in time to 2025-05-06T00:00:00.000, which is O2025050601-NRCA1_FP6-1.fits, delta time 0.132 days
/blue/adamginsburg/t.yoo/from_red/stpsf-data
path /blue/adamginsburg/t.yoo/from_red/stpsf-data
Importing and format-converting OPD from /blue/adamginsburg/t.yoo/from_red/stpsf-data/MAST_JWST_WSS_OPDs/O2025050601-NRCA1_FP6-1.fits
/blue/adamginsburg/t.yoo/from_red/stpsf-data
path /blue/adamginsburg/t.yoo/from_red/stpsf-data
Backing out SI WFE and OTE field dependence at the WF sensing field point (NRCA1_FP6)
/b

Fit source/group:   0%|          | 0/20264 [00:00<?, ?it/s]

ValueError: For one or more sources, the number of data points available to fit is less than the number of fit parameters. This could be due to a source(s) near the edge of the detector or if it has few unmasked pixels. Please check the input mask or source positions.